In [1]:
import pandas as pd 

In [2]:
df=pd.read_csv('Restaurant_Reviews.tsv',sep='\t')
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [3]:
import re
from nltk.stem import PorterStemmer
ps=PorterStemmer()
from nltk.corpus import stopwords
def clean_text(text):
    text=re.sub('[^a-zA-Z0-9]',' ',text)
    text=text.lower()
    text=text.split()
    text=[ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    return ' '.join(text)
df['Review']=df['Review'].apply(lambda x:clean_text(x))

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['Review'].values,df['Liked'].values,random_state=42,test_size=0.2,stratify=df['Liked'].values)
x_train[:5],y_train[:5]

(array(['perfect sit famili meal get togeth friend', 'servic slow attent',
        'food outstand price reason', 'two bite refus eat anymor',
        'gone go'], dtype=object),
 array([1, 0, 1, 0, 1], dtype=int64))

In [5]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import TextVectorization

In [6]:
max_tokens=1500
output_sequence_length=6

In [7]:
text_vectorizer=TextVectorization(max_tokens=max_tokens,output_sequence_length=output_sequence_length)
text_vectorizer.adapt(x_train)

In [8]:
from keras.layers import Embedding
embedding=Embedding(max_tokens,32,input_length=output_sequence_length)

In [27]:
inputs=keras.layers.Input(shape=(1,),dtype='string')
x=text_vectorizer(inputs)
x=embedding(x)
x=keras.layers.Bidirectional(keras.layers.LSTM(16,return_sequences=True))(x)
x=keras.layers.Dropout(0.2)(x)
x=keras.layers.Bidirectional(keras.layers.LSTM(16,return_sequences=True))(x)
x=keras.layers.LSTM(32,return_sequences=True)(x)
x=keras.layers.GRU(32)(x)
x=keras.layers.Dense(100,activation='relu')(x)
outputs=keras.layers.Dense(1,activation='sigmoid')(x)
model=keras.Model(inputs,outputs,name='Bidirectional')

In [28]:
model.summary()

Model: "Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 6)                0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 6, 32)             48000     
                                                                 
 bidirectional_4 (Bidirectio  (None, 6, 32)            6272      
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 6, 32)             0         
                                                                 
 bidirectional_5 (Bidirectio  (None, 6, 32)          

In [29]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
25/25 [==============================] - 9s 89ms/step - loss: 0.6455 - accuracy: 0.8600 - val_loss: 0.6031 - val_accuracy: 0.7150
Epoch 2/100
25/25 [==============================] - 0s 10ms/step - loss: 0.1627 - accuracy: 0.9925 - val_loss: 1.4301 - val_accuracy: 0.7650
Epoch 3/100
25/25 [==============================] - 0s 10ms/step - loss: 0.0185 - accuracy: 0.9950 - val_loss: 1.6963 - val_accuracy: 0.7750
Epoch 4/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0148 - accuracy: 0.9950 - val_loss: 1.4882 - val_accuracy: 0.7900
Epoch 5/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0125 - accuracy: 0.9962 - val_loss: 1.5283 - val_accuracy: 0.7650
Epoch 6/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0145 - accuracy: 0.9912 - val_loss: 1.4027 - val_accuracy: 0.7850
Epoch 7/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0164 - accuracy: 0.9962 - val_loss: 1.4250 - val_accuracy: 0.7800
Epoch 

25/25 [==============================] - 0s 11ms/step - loss: 0.0084 - accuracy: 0.9962 - val_loss: 2.2917 - val_accuracy: 0.7550
Epoch 59/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0073 - accuracy: 0.9975 - val_loss: 2.3056 - val_accuracy: 0.7650
Epoch 60/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0070 - accuracy: 0.9975 - val_loss: 2.3639 - val_accuracy: 0.7550
Epoch 61/100
25/25 [==============================] - 0s 10ms/step - loss: 0.0070 - accuracy: 0.9975 - val_loss: 2.4260 - val_accuracy: 0.7650
Epoch 62/100
25/25 [==============================] - 0s 10ms/step - loss: 0.0064 - accuracy: 0.9962 - val_loss: 2.4308 - val_accuracy: 0.7500
Epoch 63/100
25/25 [==============================] - 0s 10ms/step - loss: 0.0071 - accuracy: 0.9975 - val_loss: 2.5173 - val_accuracy: 0.7600
Epoch 64/100
25/25 [==============================] - 0s 10ms/step - loss: 0.0064 - accuracy: 0.9975 - val_loss: 2.6351 - val_accuracy: 0.7650
Epoch 65/100

In [30]:
predictions=model.predict(x_test)

In [31]:
y_pred=[]
for element in predictions:
    if element>=0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [32]:
from sklearn.metrics import *

In [33]:
confusion_matrix(y_test,y_pred)

array([[72, 28],
       [23, 77]], dtype=int64)

In [34]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.76      0.72      0.74       100
           1       0.73      0.77      0.75       100

    accuracy                           0.74       200
   macro avg       0.75      0.74      0.74       200
weighted avg       0.75      0.74      0.74       200



In [36]:
accuracy_score(y_test,y_pred)

0.745